#### Author: Arash Farahani
#### Date: 8/1/2017

### Usage
This program goes over all the Manhattan GBATed DOL-QCEW files and checks the error sheet in Google Maps API to retreive Formatted Addresses and Lat-Lon. The process is repeated twice: first with the original address, and next with the Trade/Legal name added to the address. 

#### The next steps
1. Check the final addresses manually: do the Google Formatted Addresses make sense?
2. Run through GBAT again using a final address.

In [15]:
import logging
logging.basicConfig(filename='Y:/Python Procedures/QCEW_Address_Check.log',level=logging.DEBUG)

import googlemaps
from datetime import datetime
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 60
import re, os
import configparser
import json,sys

config = configparser.RawConfigParser()
config.read('API_Keys.cfg')
google_api_key = config.get('Google', 'QCEW_API_Key')

gmaps = googlemaps.Client(key=google_api_key)

#test
geocode_result=gmaps.geocode("Indepenent Buget Office, New York, NY 10003")
print(geocode_result[0]['geometry']['location']['lat'],geocode_result[0]['geometry']['location']['lng'])
#print(geocode_result)


40.7088649 -74.00678959999999


In [16]:
# Convert JSON tree to a Python dict
#data = json.loads(geocode_result)

# print to stderr so we can verfiy that the tree is correct.
print(json.dumps(geocode_result, indent=4))



[
    {
        "address_components": [
            {
                "long_name": "# 1401",
                "short_name": "# 1401",
                "types": [
                    "subpremise"
                ]
            },
            {
                "long_name": "110",
                "short_name": "110",
                "types": [
                    "street_number"
                ]
            },
            {
                "long_name": "William Street",
                "short_name": "William St",
                "types": [
                    "route"
                ]
            },
            {
                "long_name": "Manhattan",
                "short_name": "Manhattan",
                "types": [
                    "political",
                    "sublocality",
                    "sublocality_level_1"
                ]
            },
            {
                "long_name": "New York",
                "short_name": "New York",
                "types": [
     

In [86]:
def Geocode_Qcew(Boroughs=['Manhattan','Bronx','Brooklyn','Queens','Staten Island','Unknown'],ExcludeYears=[i for i in range(2000, 2050)]):
    '''
    Description: Geocodes the QCEW records that were not found in GBAT.
    By default all the years are in the exclude list so that we don't redo any years and don't pay Google twice.
    '''
    #tuple of boros to be used in Addresses.
    Boros={1:'Manhattan',2:'Bronx',3:'Brooklyn',4:'Queens',5:'Staten Island'}
    
    for b in Boroughs:
        directory=os.fsencode("Z:/EAD/DOL Data/QCEW to RPAD address merge/forgbat/" + b +"/")
        os.chdir(directory)
        retval = os.getcwd()
        print("Current working directory: %s" % retval)
        
        for file in os.listdir(directory):
            filename = os.fsdecode(file)
            num_search = re.search('[0-9]{2}', filename, re.IGNORECASE)
            if num_search:
                num = int(num_search.group(0))+2000
            if filename.endswith(".xlsx") and '_GoogOut_' not in filename and num not in ExcludeYears:
                # Read in the GBATErr Sheet for each file.
                print("Processing: %s" % os.path.join(directory,filename.encode()))
                adds=pd.ExcelFile(os.path.join(directory,filename.encode()).decode())
                print("This Excel file includes these sheets: %s" % adds.sheet_names)
                df = adds.parse('GBATErr')
                df = df.drop(['BadRecordId','GRC','ReasonCode'], 1)   #drop the GBAT Err Columns
                print(filename, ' loaded.')
                


                df.fillna('')
                # trade2 is trade name when available, and legal name when not.
                df['trade2'] = ""
                df.loc[df.trade.fillna('')!= '', 'trade2'] = df.trade 
                df.loc[df.trade.fillna('')== '', 'trade2'] = df.legal
                # trade(or legal) name + Original Address + City, State, Zip
                if df.originaladdress.isnull:
                    df['temp_add']='' 
                else:
                    df['temp_add']=  df.originaladdress.fillna('')+', '
                df['NameAddress']= df.trade2.fillna('')+', '+df['temp_add']+df['boro'].map(Boros)+', NY '+df.pzip.apply(str)
                df['Address']= df['temp_add']+df['boro'].map(Boros)+', NY '+df.pzip.apply(str)
                df['NameAddress'].head()
                #print(df.head())
                df.drop(['temp_add'], 1)
                
                df.originaladdress=df.originaladdress.replace('*** NEED PHYSICAL ADDRESS ***','')
                df=df.fillna('')
                
                #df.reset_index(inplace=True)
                df['Gformatted_address0']= ""
                df['Glat0']= 0
                df['Glon0']= 0
                df['GPartial0']= False
                df['Gtypes0']=""
                df['Gformatted_address1']= ""
                df['Glat1']= 0
                df['Glon1']= 0
                df['GPartial1']= False
                df['Gtypes1']=""
                df['Borough0']=""
                df['Borough1']=""
                i=-1
                # Run google API twice, for the Address only and Name+Address.
                for var in ['Address','NameAddress']:
                    print('Started checking variable ',var)
                    i+=1
                    for index, row in df.iterrows():
                        #set index <= len(df.index) to process all observations.
                        if index<=len(df.index) and not(var in ['Address'] and row.originaladdress ==''):
                            geocode_result=gmaps.geocode(row[var])
                            if len(geocode_result)>0:
                                if 'partial_match' in geocode_result:
                                    df.loc[df.index == index,['GPartial'+ str(i),'Gformatted_address'+ str(i), 'Glat'+ str(i), 'Glon'+ str(i), 'Gtypes'+ str(i)]]=(
                                                         geocode_result[0]['partial_match'], geocode_result[0]['formatted_address']
                                                         ,geocode_result[0]['geometry']['location']['lat'],geocode_result[0]['geometry']['location']['lng'],str(geocode_result[0]['types']))
                                else:
                                    df.loc[df.index == index,['Gformatted_address'+ str(i), 'Glat'+ str(i), 'Glon'+ str(i), 'Gtypes'+ str(i)]]=(
                                        geocode_result[0]['formatted_address'],geocode_result[0]['geometry']['location']['lat']
                                        ,geocode_result[0]['geometry']['location']['lng'],str(geocode_result[0]['types']))
                            else:
                                df.loc[df.index == index,['Gformatted_address'+ str(i), 'Glat'+ str(i), 'Glon'+ str(i), 'Gtypes'+ str(i)]]=(
                                                                                                            'Not Found',0,0,str([0]))
                    # Prepare for Second Run of GBAT
                    df['Gzip'+ str(i)]= 0
                    pat2= r".*NY ([0-9]{5}).*"
                    repl0 = lambda m: m.group(1)
                    df['Gzip'+ str(i)]=df['Gformatted_address'+ str(i)].str.replace(pat2, repl0)

                    pat= r"([0-9\-]+)(.*?),\s(.*?)(,.*)"
                    repl1 = lambda m: m.group(1)
                    repl2 = lambda m: m.group(2)
                    repl3 = lambda m: m.group(3)
                    df['Gnumber'+ str(i)]=df['Gformatted_address'+ str(i)].str.replace(pat, repl1)
                    df['Gstreet'+ str(i)]=df['Gformatted_address'+ str(i)].str.replace(pat, repl2)
                    df['Borough'+ str(i)]=df['Gformatted_address'+ str(i)].str.replace(pat, repl3)
                    df[['Gnumber'+ str(i),'Gformatted_address'+ str(i)]].head(10)

                # Save The Results
                df['same_GF_Address']=(df['Gformatted_address1']==df['Gformatted_address0'])
                pd.ExcelFile(os.path.join(directory,filename.encode()).decode())
                writer = pd.ExcelWriter(os.path.join(directory,('_GoogOut_'+filename).encode()).decode())
                df.to_excel(writer,'Sheet1')
                print('Processed data and saved: ', '_GoogOut_'+filename )
                #df2.to_excel(writer,'Sheet2')
                writer.save()
            else:
                continue
                
#test: remember that range(2000, 2017) excludes 2017. 
Geocode_Qcew(Boroughs=['Bronx'],ExcludeYears=[i for i in range(2000, 2017)])
 

Current working directory: Z:\EAD\DOL Data\QCEW to RPAD address merge\forgbat\Bronx
Processing: b'Z:/EAD/DOL Data/QCEW to RPAD address merge/forgbat/Bronx/forgbatbx17.xlsx'
This Excel file includes these sheets: ['forgbatbx17', 'GBATOut', 'GBATErr']
forgbatbx17.xlsx  loaded.
Started checking variable  Address
Started checking variable  NameAddress
Processed data and saved:  _GoogOut_forgbatbx17.xlsx


In [275]:
### OLD CODE THAT RAN ON MANHATTAN ONLY. ###

# This is the directory of GBATed files.
directory = os.fsencode("Z:/EAD/DOL Data/QCEW to RPAD address merge/forgbat/Manhattan/")

# these files were already processed by google. I don't want to pay for processig them again, so I'll skip them.
processed_files=['forgbatmn01.xlsx','forgbatmn05.xlsx']

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".xlsx") and '_GoogOut_' not in filename and filename not in processed_files:
        # Read in the GBATErr Sheet for each file.
        print(os.path.join(directory,filename.encode()))
        adds=pd.ExcelFile(os.path.join(directory,filename.encode()).decode())
        print(adds.sheet_names)
        df = adds.parse('GBATErr')
        df = df.drop(['BadRecordId','GRC','ReasonCode'], 1)   #drop the GBAT Err Columns
        print(filename, ' was loaded')
        df.head()
        
        
        df.fillna('')
        # trade2 is trade name when available, and legal name when not.
        df['trade2'] = ""
        df.loc[df.trade.fillna('')!= '', 'trade2'] = df.trade 
        df.loc[df.trade.fillna('')== '', 'trade2'] = df.legal
        # trade(or legal) name + Original Address + City, State, Zip
        df['NameAddress']= df.trade2.fillna('')+', '+df.originaladdress.fillna('')+', New York, NY '+df.pzip.apply(str)
        df['Address']= df.originaladdress.fillna('')+', New York, NY '+df.pzip.apply(str)
        df['NameAddress'].head()
        
        #df.reset_index(inplace=True)
        df['Gformatted_address0']= ""
        df['Glat0']= 0
        df['Glon0']= 0
        df['GPartial0']= False
        df['Gtypes0']=""
        df['Gformatted_address1']= ""
        df['Glat1']= 0
        df['Glon1']= 0
        df['GPartial1']= False
        df['Gtypes1']=""
        i=-1
        # Run google API twice, for the Address only and Name+Address.
        for var in ['Address','NameAddress']:
            print('Started checking variable ',var)
            i+=1
            for index, row in df.iterrows():
                if index<=len(df.index) and Address ~= ', New York, NY '+df.pzip.apply(str):
                    geocode_result=gmaps.geocode(row[var])
                    if len(geocode_result)>0:
                        if 'partial_match' in geocode_result:
                            df.loc[df.index == index,['GPartial'+ str(i),'Gformatted_address'+ str(i), 'Glat'+ str(i), 'Glon'+ str(i), 'Gtypes'+ str(i)]]=(
                                                 geocode_result[0]['partial_match'], geocode_result[0]['formatted_address']
                                                 ,geocode_result[0]['geometry']['location']['lat'],geocode_result[0]['geometry']['location']['lng'],str(geocode_result[0]['types']))
                        else:
                            df.loc[df.index == index,['Gformatted_address'+ str(i), 'Glat'+ str(i), 'Glon'+ str(i), 'Gtypes'+ str(i)]]=(
                                geocode_result[0]['formatted_address'],geocode_result[0]['geometry']['location']['lat']
                                ,geocode_result[0]['geometry']['location']['lng'],str(geocode_result[0]['types']))
                    else:
                        df.loc[df.index == index,['Gformatted_address'+ str(i), 'Glat'+ str(i), 'Glon'+ str(i), 'Gtypes'+ str(i)]]=(
                                                                                                    'Not Found',0,0,str([0]))
            # Prepare for Second Run of GBAT
            df['Gzip'+ str(i)]= 0
            pat2= r".*([0-9]{5}).*"
            repl0 = lambda m: m.group(1)
            df['Gzip1']=df['Gformatted_address'+ str(0)].str.replace(pat, repl0)
            
            pat= r"([0-9\-]+)(.*?)(,.*)"
            repl1 = lambda m: m.group(1)
            repl2 = lambda m: m.group(2)
            df['Gnumber'+ str(i)]=df['Gformatted_address'+ str(i)].str.replace(pat, repl1)
            df['Gstreet'+ str(i)]=df['Gformatted_address'+ str(i)].str.replace(pat, repl2)
            df['boro']=1
            df[['Gnumber'+ str(i),'Gformatted_address'+ str(i)]].head(10)
            
        # Save The Results
        df['same_GF_Address']=(df['Gformatted_address1']==df['Gformatted_address0'])
        pd.ExcelFile(os.path.join(directory,filename.encode()).decode())
        writer = pd.ExcelWriter(os.path.join(directory,('_GoogOut_'+filename).encode()).decode())
        df.to_excel(writer,'Sheet1')
        print('Processed data and saved: ', '_GoogOut_'+filename )
        #df2.to_excel(writer,'Sheet2')
        writer.save()
    else:
        continue


## SOME OTHER STUFF
# Look up an address with reverse geocoding
#reverse_geocode_result = gmaps.reverse_geocode((40.714224, -73.961452))

# Request directions via public transit
#now = datetime.now()
#directions_result = gmaps.directions("Sydney Town Hall",
#                                     "Parramatta, NSW",
#                                     mode="transit",
#                                     departure_time=now)

## Some Other Tests
#geocode_result=gmaps.geocode("BETH ISRAEL MEDICAL CENTER, 1ST AVE AT 16TH ST, New York, NY 10003")
#print(geocode_result[0]['geometry']['location']['lat'],geocode_result[0]['geometry']['location']['lng'])
#print(geocode_result[0]['partial_match'])
#print(geocode_result[0])



b'Z:/EAD/DOL Data/QCEW to RPAD address merge/forgbat/Manhattan/forgbatmn07.xlsx'
['forgbatmn07', 'GBATOut', 'GBATErr', 'GBATExt']
forgbatmn07.xlsx  was loaded
Started checking variable  Address
Started checking variable  NameAddress
Processed data and saved:  _GoogOut_forgbatmn07.xlsx
b'Z:/EAD/DOL Data/QCEW to RPAD address merge/forgbat/Manhattan/forgbatmn11.xlsx'
['forgbatmn11', 'GBATOut', 'GBATErr']
forgbatmn11.xlsx  was loaded
Started checking variable  Address
Started checking variable  NameAddress
Processed data and saved:  _GoogOut_forgbatmn11.xlsx
b'Z:/EAD/DOL Data/QCEW to RPAD address merge/forgbat/Manhattan/forgbatmn02.xlsx'
['forgbatmn02', 'GBATOut', 'GBATErr']
forgbatmn02.xlsx  was loaded
Started checking variable  Address
Started checking variable  NameAddress
Processed data and saved:  _GoogOut_forgbatmn02.xlsx
b'Z:/EAD/DOL Data/QCEW to RPAD address merge/forgbat/Manhattan/forgbatmn13.xlsx'
['forgbatmn13', 'GBATOut', 'GBATErr']
forgbatmn13.xlsx  was loaded
Started checking

In [34]:
df.head()

,pstreet1,streetnumber,streetname,er,run,legal,trade,originaladdress,totwages05a,avgemp05a,naics1,naics2,naics3,naics4,pzip,meei,own,boro,recordid
0,LUBIN HOUSE,LUBIN,NaN,450375,3,SYRACUSE UNIVERSITY,SYRACUSE UNIVERSITY,LUBIN HOUSE,870672,15.333333,551114,551114,551114,551114,10021,3,5,1,57
1,480 MAPLE STREET,480,MAPLE STREET,450434,109,JBFCS,MAPLE HOUSE,480 MAPLE STREET,461034,14.000000,623220,623220,623220,623220,10018,3,5,1,91
2,3 DAG HAMMARSKJOLD PLAZA,3,DAG HAMMARSKJOLD PLAZA,450664,0,AMERICAN UNIVERSITY OF BEIRUT,AMERICAN UNIVERSITY OF BEIRUT,3 DAG HAMMARSKJOLD PLAZA,0,0.000000,813219,813219,813219,813219,10017,1,5,1,179
3,55 WALER ST,55,WALER STREET,450682,0,THE ROBERT BOWNE,THE ROBERT BOWNE,55 WALER ST,350689,11.833333,813211,813211,813211,813211,10041,1,5,1,182
4,F10/F80/F85/F90/F95/F70,F10,NaN,451391,10,GREENWICH HOUSE INC,GREENWICH HOUSE INC,F10/F80/F85/F90/F95/F70,1055596,33.250000,624190,624190,624190,624190,10011,3,5,1,323


In [276]:
print(len(df.index))

2610


### Process Results for further cleaning
At the moment (8/2/17) I just check the records that are in my CRP-2005 area zipcodes and have total wages ge 200k.

In [304]:
# Add Google zip-codes to files that were processed previoiusly

directory = os.fsencode("Z:/EAD/DOL Data/QCEW to RPAD address merge/forgbat/Manhattan/")

# these files were already processed by google. I don't want to pay for processig them again, so I'll edit them.
exclude_files=['_GoogOut_forgbatmn01 - edited.xlsx']

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".xlsx") and '_GoogOut_' in filename and filename not in exclude_files:
        adds=pd.ExcelFile(os.path.join(directory,filename.encode()).decode())
        print(filename, ' to load')
        print('sheet names: ',adds.sheet_names)        
        df = adds.parse('Sheet1')      
        print(filename, ' was loaded')
        
        for i in [0,1]:
            df['Gzip'+ str(i)]= 0
            pat2= r".*NY ([0-9]{5}).*"
            repl0 = lambda m: m.group(1)
            df['Gzip'+ str(i)]=df['Gformatted_address'+ str(i)].str.replace(pat2, repl0)
        
        
        # Save The Results
        df['same_GF_Address']=(df['Gformatted_address1']==df['Gformatted_address0'])
        pd.ExcelFile(os.path.join(directory,filename.encode()).decode())
        writer = pd.ExcelWriter(os.path.join(directory,(filename).encode()).decode())
        df.to_excel(writer,'Sheet1')
        print('Processed data and saved: ', filename )
        #df2.to_excel(writer,'Sheet2')
        writer.save()
df.head()


_GoogOut_forgbatmn01.xlsx  to load
sheet names:  ['Sheet1']
_GoogOut_forgbatmn01.xlsx  was loaded
Processed data and saved:  _GoogOut_forgbatmn01.xlsx


,pstreet1,streetnumber,streetname,er,run,legal,trade,originaladdress,totwages01a,avgemp01a,...,Glat1,Glon1,GPartial1,Gtypes1,Gnumber0,Gstreet0,Gnumber1,Gstreet1,Gzip0,Gzip1
762,555 W 5TH ST,555,WEST 5 STREET,1851403,0,HEALTHFIRST INC,HEALTHFIRST INC,555 W 5TH ST,33299525,686.000000,...,0.000000,0.000000,False,[0],555,W 5th St,Not Found,Not Found,13126,Not Found
1126,260W 44TH ST,260,44 STREET,3881763,0,MSX INTERNATIONAL ENGINEERING,MSX INTERNATIONAL ENGINEERING,260W 44TH ST,18050605,305.000000,...,0.000000,0.000000,False,[0],260,W 44th St,Not Found,Not Found,10036,Not Found
1294,MORGAN STANLEY VIP,MORGAN,NaN,4507308,6,CORESTAFF ASSOCIATES LLC,CORESTAFF ASSOCIATES LLC,MORGAN STANLEY VIP,12400216,450.666667,...,40.761012,-73.978157,False,"['establishment', 'finance', 'point_of_interest']",31,West 52nd Street,31,West 52nd Street,10019,10019
1904,#5 HANOVER SQUARE 6TH FL,#5,NaN,6370321,9,RELIANCE INSURANCE CO,RELIANCE INSURANCE CO,#5 HANOVER SQUARE 6TH FL,10488659,110.111111,...,0.000000,0.000000,False,[0],5,Hanover Square,Not Found,Not Found,10004,Not Found
1220,DELAWARE INC ETAL,DELAWA,NaN,4292311,0,SANWA FINANCIAL PRODUCTS,SANWA FINANCIAL PRODUCTS,DELAWARE INC ETAL,10301946,16.666667,...,0.000000,0.000000,False,[0],Not Found,Not Found,Not Found,Not Found,Not Found,Not Found


In [305]:
target_zips=[10001,10002,10004,10005,10006,10007,10013,10018,10038,10041,10048,10119,10123]

directory = os.fsencode("Z:/EAD/DOL Data/QCEW to RPAD address merge/forgbat/Manhattan/")

# these files were already processed by google. I don't want to pay for processig them again, so I'll skip them.
incomplete_files=['_GoogOut_forgbatmn01 - edited.xlsx']

# Clean up the files for processing 

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".xlsx") and '_GoogOut_' in filename and filename not in incomplete_files:
        adds=pd.ExcelFile(os.path.join(directory,filename.encode()).decode())
        print(filename, ' to load')
        print('sheet names: ',adds.sheet_names)        
        df = adds.parse('Sheet1')
        df['zip_keep'] = (df.Gzip0.isin(target_zips)) | (df.Gzip1.isin(target_zips)) | (df.pzip.isin(target_zips))   #drop the GBAT Err Columns
        print(filename, ' was loaded')
        
        # Save The Results
        df['same_GF_Address']=(df['Gformatted_address1']==df['Gformatted_address0'])
        pd.ExcelFile(os.path.join(directory,filename.encode()).decode())
        writer = pd.ExcelWriter(os.path.join(directory,(filename).encode()).decode())
        df.to_excel(writer,'Sheet1')
        print('Processed data and saved: ', filename )
        #df2.to_excel(writer,'Sheet2')
        writer.save()


_GoogOut_forgbatmn11.xlsx  to load
sheet names:  ['Sheet1']
_GoogOut_forgbatmn11.xlsx  was loaded
Processed data and saved:  _GoogOut_forgbatmn11.xlsx
_GoogOut_forgbatmn06.xlsx  to load
sheet names:  ['Sheet1']
_GoogOut_forgbatmn06.xlsx  was loaded
Processed data and saved:  _GoogOut_forgbatmn06.xlsx
_GoogOut_forgbatmn01.xlsx  to load
sheet names:  ['Sheet1']
_GoogOut_forgbatmn01.xlsx  was loaded
Processed data and saved:  _GoogOut_forgbatmn01.xlsx
_GoogOut_forgbatmn05.xlsx  to load
sheet names:  ['Sheet1']
_GoogOut_forgbatmn05.xlsx  was loaded
Processed data and saved:  _GoogOut_forgbatmn05.xlsx
_GoogOut_forgbatmn13.xlsx  to load
sheet names:  ['Sheet1']
_GoogOut_forgbatmn13.xlsx  was loaded
Processed data and saved:  _GoogOut_forgbatmn13.xlsx
_GoogOut_forgbatmn10.xlsx  to load
sheet names:  ['Sheet1']
_GoogOut_forgbatmn10.xlsx  was loaded
Processed data and saved:  _GoogOut_forgbatmn10.xlsx
_GoogOut_forgbatmn00.xlsx  to load
sheet names:  ['Sheet1']
_GoogOut_forgbatmn00.xlsx  was loa

In [308]:
df.head()

,pstreet1,streetnumber,streetname,er,run,legal,trade,originaladdress,totwages08a,avgemp08a,naics1,naics2,naics3,naics4,pzip,meei,own,boro,recordid,trade2,NameAddress,Address,Gformatted_address0,Glat0,Glon0,GPartial0,Gtypes0,Gformatted_address1,Glat1,Glon1,GPartial1,Gtypes1,Gzip0,Gzip1,Gnumber0,Gstreet0,Gnumber1,Gstreet1,same_GF_Address,zip_keep
0,480 MAPLE STREET,480,MAPLE STREET,450434,109,JBFCS,MAPLE HOUSE,480 MAPLE STREET,481634,14.000000,623220,623220,623220,623220,10018,3,5,1,95,MAPLE HOUSE,"MAPLE HOUSE, 480 MAPLE STREET, New York, NY 10018","480 MAPLE STREET, New York, NY 10018","480 Maple St, Brooklyn, NY 11225, USA",40.661025,-73.944682,False,['premise'],"480 Maple St, Brooklyn, NY 11225, USA",40.661022,-73.944692,False,"['establishment', 'health', 'point_of_interest']",11225,11225,480,Maple St,480,Maple St,True,True
1,PS 154,PS,NaN,451039,4,FOOD BANK FOR NEW YORK CITY,NaN,PS 154,4123,2.000000,0,0,0,624120,10027,3,5,1,280,FOOD BANK FOR NEW YORK CITY,"FOOD BANK FOR NEW YORK CITY, PS 154, New York,...","PS 154, New York, NY 10027","250 W 127th St, New York, NY 10027, USA",40.810470,-73.948712,False,"['establishment', 'point_of_interest', 'school']","250 W 127th St, New York, NY 10027, USA",40.810470,-73.948712,False,"['establishment', 'point_of_interest', 'school']",10027,10027,250,W 127th St,250,W 127th St,True,False
2,PS 134,PS,NaN,451039,5,FOOD BANK FOR NEW YORK CITY,NaN,PS 134,2603,1.000000,0,0,0,624120,10002,3,5,1,281,FOOD BANK FOR NEW YORK CITY,"FOOD BANK FOR NEW YORK CITY, PS 134, New York,...","PS 134, New York, NY 10002","293 E Broadway, New York, NY 10002, USA",40.714301,-73.982966,False,"['establishment', 'point_of_interest', 'school']","39 Broadway, New York, NY 10038, USA",40.706356,-74.013422,False,"['establishment', 'point_of_interest']",10002,10038,293,E Broadway,39,Broadway,False,True
3,522 W 173 ST,522,WEST 173 STREET,452470,9,COMUNILIFE INC,NaN,522 W 173 ST,236506,8.416667,624221,624221,624221,624221,10016,3,5,1,454,COMUNILIFE INC,"COMUNILIFE INC, 522 W 173 ST, New York, NY 10016","522 W 173 ST, New York, NY 10016","522 W 173rd St, New York, NY 10032, USA",40.843289,-73.935492,False,['street_address'],"315 E 148th St, Bronx, NY 10451, USA",40.816325,-73.921244,False,"['establishment', 'health', 'point_of_interest']",10032,10451,522,W 173rd St,315,E 148th St,False,False
4,139 141 AVE D,139,141 AVENUE,454798,12,BRC HUMAN SERVICES CORP,BOWERY RESIDENTS COMMITTEE INC,139-141 AVE D,568347,17.500000,623990,623990,623990,623990,10009,3,5,1,684,BOWERY RESIDENTS COMMITTEE INC,"BOWERY RESIDENTS COMMITTEE INC, 139-141 AVE D,...","139-141 AVE D, New York, NY 10009","141 Avenue D, New York, NY 10009, USA",40.724677,-73.975626,False,['street_address'],"139 Avenue D, New York, NY 10009, USA",40.724684,-73.975540,False,"['establishment', 'health', 'point_of_interest']",10009,10009,141,Avenue D,139,Avenue D,False,False


### Carry forward the Manual address checks 
At this stage, I have ran a VBA macro on the excel files outputted above to filter the records that I want to clean up.
Finally, I have gone over those records and created a Final Address column. Those addresses are verified or marked as Not Found.

The filters as a of 8/3/17 were being in the zipcode for my programs (noted above) and having annual total wages above $500K.

In [58]:
directory = os.fsencode("Z:/EAD/DOL Data/QCEW to RPAD address merge/forgbat/Manhattan/Google_Out_processed by macro/")

# these files were already processed by google. I don't want to pay for processig them again, so I'll skip them.
input_files=['_GoogOut_forgbatmn00.xlsx','_GoogOut_forgbatmn01.xlsx','_GoogOut_forgbatmn02.xlsx']

# Clean up the files for processing 

for file in os.listdir(directory):
    #print(os.path.splitext(file)[0].decode())
    filename = os.fsdecode(file)
    if filename.endswith(".xlsx") and '_GoogOut_' in filename and filename in input_files:
        adds=pd.ExcelFile(os.path.join(directory,filename.encode()).decode())
        print(filename, ' to load')
        print('sheet names: ',adds.sheet_names)        
        df = adds.parse('Sheet1')
        if 'Final Address' not in df.columns:
            df['Final Address']=''
        df1 = df.loc[df['Final Address']!="Not Searched",['Final Address','trade2', 'Address']]
        name= os.path.splitext(file)[0].decode()
        name1= name + '_adds'
        exec(name +'= df')
        exec(name +'= df1')
        print(filename, ' was loaded')
        
_GoogOut_forgbatmn00 = _GoogOut_forgbatmn00.loc[_GoogOut_forgbatmn00['Final Address']!="Not Searched",['Final Address','trade2', 'Address']]


new_df = pd.merge(_GoogOut_forgbatmn01, _GoogOut_forgbatmn00,  how='left', left_on=['trade2','Address'], right_on = ['trade2','Address'],indicator =True)


_GoogOut_forgbatmn01.xlsx  to load
sheet names:  ['Sheet1', 'GBATOut', 'GBATErr']
_GoogOut_forgbatmn01.xlsx  was loaded
_GoogOut_forgbatmn00.xlsx  to load
sheet names:  ['Sheet1', 'GBATOut', 'GBATErr']
_GoogOut_forgbatmn00.xlsx  was loaded
_GoogOut_forgbatmn02.xlsx  to load
sheet names:  ['Sheet1', 'GBATOut', 'GBATErr']
_GoogOut_forgbatmn02.xlsx  was loaded


In [52]:
_GoogOut_forgbatmn00.count()

Final Address    67
trade2           67
Address          67
dtype: int64

In [56]:
new_df02 = pd.merge(_GoogOut_forgbatmn02, _GoogOut_forgbatmn00,  how='outer', left_on=['trade2','Address'], right_on = ['trade2','Address'],indicator =True)
pd.crosstab(new_df._merge,new_df.zip_keep)
pd.crosstab(new_df02._merge,new_df02.zip_keep)


zip_keep,0.0,1.0
_merge,,
left_only,1900,641
right_only,0,0
both,0,19


In [16]:
df.head()

,,Final Address,trade2,Address
1664.0,1664.0,Not Found,FLEET SECURITIES INC,"WALL ST, New York, NY 10041"
2256.0,2256.0,"1 Battery Plaza, New York, NY 10004, USA",SEWARD & KISSEL LLP,"1 BATTERY PARK PL 21ST FL, New York, NY 10004"
329.0,329.0,"1515 Broadway, New York, NY 10036",VIACOM SERVICES CORP,"57TH ST, New York, NY 10001"
1145.0,1145.0,"401 Broadway, New York, NY 10013, USA",JUDLAU CONTRACTING INC,"WALL ST, New York, NY 10005"
1767.0,1767.0,"470 7th Ave, New York, NY 10018, USA",BUGLE BOY 'CORP',"SUITE #401,405,501, New York, NY 10018"
